In [ ]:
# https://towardsdatascience.com/the-4-recommendation-engines-that-can-predict-your-movie-tastes-109dc4e10c52  
# 1 https://github.com/khanhnamle1994/movielens/blob/master/Deep_Learning_Model.ipynb
# 2 CFModel.py https://github.com/khanhnamle1994/movielens/blob/master/CFModel.py 
# 3 https://github.com/chen0040/keras-recommender/blob/master/keras_recommender/library/cf.py#L57
# datasets : movielens 1M dataset

# This code is a combination of 1, 2 and 3.

# 2019.04.28.SUN.20:10. by kwangsuk.lee
# 2019.05.01.WEN.21:10.
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Import Keras libraries
import keras 
from keras.models import Model, Sequential  # New 
from keras.layers import Input, Embedding, Reshape, dot, Concatenate, Flatten, Dropout, Dense # delete : Merge,  
from keras.layers import Add, concatenate, Activation, BatchNormalization, regularizers
from keras.layers import LSTM, Bidirectional 
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
# The keras.layers.merge layer is deprecated. Use keras.layers.Concatenate(axis=-1) 
# instead as mentioned here: https://keras.io/layers/merge/#concatenate
# keras Merge = Concatenate     by kslee
# keras Concatenate dot = dot   by kslee
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error #Mean squared error regression loss
from sklearn.metrics import mean_absolute_error # Mean absolute error regression lossv
from keras.utils import plot_model

# New coding by kwangsuk.lee
ModelName = 'DLmodel_9'

EMBEDDING_SIZE = 100  # num of token(vaca=word), 단어 갯수
VERBOSE = 1
EPOCHES = 20 # 20, 30 
BATCH_SIZE=32 # 32, 64 bad, 128 bad 


# Reading ratings file  , .csv', sep='\t', 
ratings = pd.read_csv('./ml-1m/ratings.csv', sep='\t', encoding='utf-8', 
                      usecols=['user_id', 'movie_id', 'rating', 'user_emb_id', 'movie_emb_id'])

max_user_id = ratings['user_id'].drop_duplicates().max()

#max_movie_id = ratings['movie_id'].drop_duplicates().max()
max_item_id = ratings['movie_id'].drop_duplicates().max()

# Reading users file , .csv', sep='\t', 
users = pd.read_csv('./ml-1m/users.csv', sep='\t', encoding='latin-1', 
                    usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file , .csv', sep='\t', 
movies = pd.read_csv('./ml-1m/movies.csv', sep='\t', encoding='latin-1', 
                     usecols=['movie_id', 'title', 'genres'])

RANDOM_SEED = 5  # New, by kwangsuk.lee

# Create training set
shuffled_ratings = ratings.sample(frac=1., random_state=RANDOM_SEED)

# Shuffling users
Users = shuffled_ratings['user_emb_id'].values
print('Users:', Users, ', shape =', Users.shape)

# Shuffling movies
Movies = shuffled_ratings['movie_emb_id'].values
print('Movies:', Movies, ', shape =', Movies.shape)

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print('Ratings:', Ratings, ', shape =', Ratings.shape)

# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
TEST_USER = 2000 # A random test user (user_id = 2000)


import keras.backend.tensorflow_backend as K 
with K.tf.device('/gpu:0'):

    user_id_input = Input(shape=[1], name='user')
    item_id_input = Input(shape=[1], name='item')
    
    user_embedding = Embedding(output_dim=EMBEDDING_SIZE, input_dim=max_user_id + 1,
                               input_length=1, name='user_embedding')(user_id_input)
                               # input_dim = max_user_id 

    item_embedding = Embedding(output_dim=EMBEDDING_SIZE, input_dim=max_item_id + 1,
                               input_length=1, name='item_embedding')(item_id_input)
                               # input_dim = max_item_id

    # = Embedding(input_dim,) 
    # input_dim: This is the size of the vocabulary in the text data.
    # output_dim: This is the size of the vector space in which words will be embedded.
    # input_length: This is the length of input sequences, as you would define for any input layer.

    # reshape from shape: (batch_size, input_length, embedding_size)
    # to shape: (batch_size, input_length * embedding_size) which is
    # equal to shape: (batch_size, embedding_size)

    
    conv_11 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu')(user_embedding)
    conv_11_out = Dropout(0.5)(conv_11)
    conv_12 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu')(conv_11_out)
    conv_12_out = Dropout(0.5)(conv_12)   
    
    bi_11 = (Bidirectional(LSTM(4, return_sequences=True, dropout=0.5)))(conv_12_out) 
    bi_12 = (Bidirectional(LSTM(8, return_sequences=True, dropout=0.5)))(bi_11)  
    
    residual_11 = keras.layers.Add()([conv_12_out, bi_12]) # 16, 16
    
    lstm_11 = (LSTM(16, return_sequences=True, dropout=0.5))(residual_11) #(bi_12) 
    lstm_12 = (LSTM(16, return_sequences=True, dropout=0.5))(lstm_11) 
    # bad, dense_11 = Dense(8, activation='relu')(lstm_12)
    # bad, out_1 = Dense(1, activation='relu')(dense_11)
    
    residual_12 = keras.layers.Add()([residual_11, lstm_12]) # 16, 16
    lstm_13 = (LSTM(8, return_sequences=True))(residual_12) #(lstm_12) 

    
    conv_21 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu')(item_embedding)
    conv_21_out = Dropout(0.5)(conv_21)
    conv_22 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu')(conv_21_out)
    conv_22_out = Dropout(0.5)(conv_22)   

    bi_11 = (Bidirectional(LSTM(4, return_sequences=True, dropout=0.5)))(conv_22_out) 
    bi_12 = (Bidirectional(LSTM(8, return_sequences=True, dropout=0.5)))(bi_11)  
    
    residual_21 = keras.layers.Add()([conv_22_out, bi_12]) # 16, 16

    lstm_21 = (LSTM(16, return_sequences=True, dropout=0.5))(residual_21) #(bi_12)   
    lstm_22 = (LSTM(16, return_sequences=True, dropout=0.5))(lstm_21) 
    # bad, dense_21 = Dense(8, activation='relu')(lstm_22)
    # bad, out_2 = Dense(1, activation='relu')(dense_21)

    residual_22 = keras.layers.Add()([residual_21, lstm_12]) # 16, 16
    lstm_23 = (LSTM(8, return_sequences=True))(residual_22) #(lstm_22) 
    
    
    user_vecs = Flatten()(lstm_13) # user_embedding  
    item_vecs = Flatten()(lstm_23) # item_embedding 

    #dotproduct = dot([user_vecs, item_vecs], axes=1, normalize=False)  # axes=1 
    dotproduct = dot([user_vecs, item_vecs], axes=1, normalize=False)  # axes=1 

    model = Model(inputs=[user_id_input, item_id_input], outputs=[dotproduct])    

    optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, amsgrad=False) 

    # model.compile() in def create_model(self): 
    # Compile the model using MSE as the loss function and the Adam learning algorithm
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae', 'mape']) # optimizer='adam'

    # Callbacks monitor the validation loss
    # Save the model weights each time the validation loss has improved
    callbacks = [EarlyStopping('val_loss', patience=4), 
                 ModelCheckpoint(ModelName+'_weights.h5', save_best_only=True), 
                 ReduceLROnPlateau(monitor='val_loss', factor = 0.1, patience=1), 
                 CSVLogger(ModelName + 'log.csv', append=True, separator='\n')]

    # Use 20 epochs, 90% training data, 10% validation data    
    history = model.fit([Users, Movies], Ratings, epochs=EPOCHES,
                        validation_split=.1, verbose=VERBOSE, batch_size=BATCH_SIZE,
                        shuffle=False, callbacks=callbacks) #batch_size=BATCH_SIZE,

    
    # Show the best validation RMSE
    min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
    print('Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss)))
    print('The end of training')
    
# model summary by text     
print('model summary ... ', model.summary())  # keras.models.model()

# model summary by plot graph
plot_model(model, to_file=ModelName+'model_image.png')  # from keras.utils import plot_model 

Using TensorFlow backend.


Users: [1578 5626 3779 ... 3357  145 1763] , shape = (1000209,)
Movies: [3267 2022  615 ...  317 2290  561] , shape = (1000209,)
Ratings: [1 3 2 ... 5 4 1] , shape = (1000209,)
Train on 900188 samples, validate on 100021 samples
Epoch 1/20
900188/900188 [==============================] - 953s 1ms/step - loss: 1.2513 - mean_squared_error: 1.2513 - mean_absolute_error: 0.8542 - mean_absolute_percentage_error: 33.1239 - val_loss: 0.8721 - val_mean_squared_error: 0.8721 - val_mean_absolute_error: 0.7397 - val_mean_absolute_percentage_error: 30.1466
Epoch 2/20
900188/900188 [==============================] - 962s 1ms/step - loss: 0.8900 - mean_squared_error: 0.8900 - mean_absolute_error: 0.7524 - mean_absolute_percentage_error: 29.8041 - val_loss: 0.8570 - val_mean_squared_error: 0.8570 - val_mean_absolute_error: 0.7354 - val_mean_absolute_percentage_error: 29.5431
Epoch 3/20
900188/900188 [==============================] - 959s 1ms/step - loss: 0.8747 - mean_squared_error: 0.8747 - mean_ab

In [2]:
# The rate function to predict user's rating of unrated items
def rate(user_id, movie_id):
    return model.predict([np.array([user_id]), np.array([movie_id])])[0][0]
#return model.predict()

# Use the pre-trained model
# trained_model = create_model(max_user_id, max_item_id)
# Load weights
# trained_model.load_weights(ModelName+'_weights.h5')
model.load_weights(ModelName+'_weights.h5')

In [3]:
# Pick a random test user
users[users['user_id'] == TEST_USER]

,user_id,gender,zipcode,age_desc,occ_desc
1999,2000,M,44685,18-24,college/grad student


In [4]:
# Function to predict the ratings given User ID and Movie ID
# call rate() in create_model()
def predict_rating(user_id, movie_id):
    #return trained_model.rate(user_id - 1, movie_id - 1)
    return rate(user_id - 1, movie_id - 1)

user_ratings = ratings[ratings['user_id'] == TEST_USER][['user_id', 'movie_id', 'rating']]

# call predict_rating
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)

user_ratings.sort_values(by='rating',ascending=False).merge(movies, 
                                                            on='movie_id', 
                                                            how='inner', 
                                                            suffixes=['_u', '_m']).head(20)


,user_id,movie_id,rating,prediction,title,genres
0,2000,1639,5,3.848909,Chasing Amy (1997),Drama|Romance
1,2000,2529,5,3.748759,Planet of the Apes (1968),Action|Sci-Fi
2,2000,1136,5,4.346476,Monty Python and the Holy Grail (1974),Comedy
3,2000,2321,5,3.924752,Pleasantville (1998),Comedy
4,2000,2858,5,4.213957,American Beauty (1999),Comedy|Drama
5,2000,2501,5,4.234559,October Sky (1999),Drama
6,2000,2804,5,4.211183,"Christmas Story, A (1983)",Comedy|Drama
7,2000,1688,5,3.555718,Anastasia (1997),Animation|Children's|Musical
8,2000,1653,5,3.673203,Gattaca (1997),Drama|Sci-Fi|Thriller
9,2000,527,5,4.460137,Schindler's List (1993),Drama|War


In [5]:
# Recommend Movies
# Here I make a recommendation list of unrated 20 movies sorted by prediction value for user 2000. Let's see it.

recommendations = ratings[ratings['movie_id'].isin(user_ratings['movie_id']) == False][['movie_id']].drop_duplicates()

recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)

recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movie_id',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)


,movie_id,prediction,title,genres
0,745,4.406321,"Close Shave, A (1995)",Animation|Comedy|Thriller
1,904,4.403423,Rear Window (1954),Mystery|Thriller
2,2019,4.396366,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
3,720,4.393840,Wallace & Gromit: The Best of Aardman Animatio...,Animation
4,1207,4.387823,To Kill a Mockingbird (1962),Drama
5,2905,4.385592,Sanjuro (1962),Action|Adventure
6,1262,4.383455,"Great Escape, The (1963)",Adventure|War
7,1250,4.381680,"Bridge on the River Kwai, The (1957)",Drama|War
8,1148,4.381193,"Wrong Trousers, The (1993)",Animation|Comedy
9,1178,4.376578,Paths of Glory (1957),Drama|War
